In [117]:
#!pip install yfinance -q
#!pip install yahoo_fin -q

In [118]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf

#extracting data from Yahoo Finance API
tickers = ['UBI.PA']
all_data = pd.DataFrame()
test_data = pd.DataFrame()
no_data = []

for i in tickers:
    try:
        test_data = yf.download(tickers=i, start='2022-04-15', interval='1h')
        test_data['symbol'] = i
        all_data = all_data.append(test_data)
    except:
        no_data.append(i)

# Lowercase column names
all_data.rename(columns={"Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume":"volume"}, inplace=True)

#Creating Return column
all_data['return'] = all_data.groupby('symbol')['close'].pct_change() 

[*********************100%***********************]  1 of 1 completed


In [119]:
all_data.shape

(753, 8)

## SMA

In [120]:
all_data['SMA_5'] = all_data.groupby('symbol')['close'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA_15'] = all_data.groupby('symbol')['close'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_ratio'] = all_data['SMA_15'] / all_data['SMA_5']

In [121]:
all_data['SMA5_Volume'] = all_data.groupby('symbol')['volume'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA15_Volume'] = all_data.groupby('symbol')['volume'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_Volume_Ratio'] = all_data['SMA5_Volume']/all_data['SMA15_Volume']

## Stochastic

In [122]:
all_data['Lowest_5D'] = all_data.groupby('symbol')['low'].transform(lambda x: x.rolling(window = 5).min())
all_data['High_5D'] = all_data.groupby('symbol')['high'].transform(lambda x: x.rolling(window = 5).max())
all_data['Lowest_15D'] = all_data.groupby('symbol')['low'].transform(lambda x: x.rolling(window = 15).min())
all_data['High_15D'] = all_data.groupby('symbol')['high'].transform(lambda x: x.rolling(window = 15).max())

all_data['Stochastic_5'] = ((all_data['close'] - all_data['Lowest_5D'])/(all_data['High_5D'] - all_data['Lowest_5D']))*100
all_data['Stochastic_15'] = ((all_data['close'] - all_data['Lowest_15D'])/(all_data['High_15D'] - all_data['Lowest_15D']))*100

all_data['Stochastic_%D_5'] = all_data['Stochastic_5'].rolling(window = 5).mean()
all_data['Stochastic_%D_15'] = all_data['Stochastic_5'].rolling(window = 15).mean()

all_data['Stochastic_Ratio'] = all_data['Stochastic_%D_5']/all_data['Stochastic_%D_15']

## RSI

In [123]:
all_data['Diff'] = all_data.groupby('symbol')['close'].transform(lambda x: x.diff())
all_data['Up'] = all_data['Diff']
all_data.loc[(all_data['Up']<0), 'Up'] = 0

all_data['Down'] = all_data['Diff']
all_data.loc[(all_data['Down']>0), 'Down'] = 0 
all_data['Down'] = abs(all_data['Down'])

all_data['avg_5up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=5).mean())
all_data['avg_5down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=5).mean())

all_data['avg_15up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=15).mean())
all_data['avg_15down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=15).mean())

all_data['RS_5'] = all_data['avg_5up'] / all_data['avg_5down']
all_data['RS_15'] = all_data['avg_15up'] / all_data['avg_15down']

all_data['RSI_5'] = 100 - (100/(1+all_data['RS_5']))
all_data['RSI_15'] = 100 - (100/(1+all_data['RS_15']))

all_data['RSI_ratio'] = all_data['RSI_5']/all_data['RSI_15']

In [124]:
all_data['5Ewm'] = all_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span=5, adjust=False).mean())
all_data['15Ewm'] = all_data.groupby('symbol')['close'].transform(lambda x: x.ewm(span=15, adjust=False).mean())
all_data['MACD'] = all_data['15Ewm'] - all_data['5Ewm']

In [125]:
all_data

,open,high,low,close,Adj Close,volume,symbol,return,SMA_5,SMA_15,...,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD
2022-04-19 09:00:00+02:00,36.700001,36.700001,35.639999,36.049999,36.049999,0,UBI.PA,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.049999,36.049999,0.000000
2022-04-19 10:00:00+02:00,36.049999,36.500000,36.020000,36.270000,36.270000,72824,UBI.PA,0.006103,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.123333,36.077499,-0.045834
2022-04-19 11:00:00+02:00,36.279999,36.279999,35.720001,35.790001,35.790001,21534,UBI.PA,-0.013234,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.012222,36.041562,0.029340
2022-04-19 12:00:00+02:00,35.770000,36.080002,35.590000,35.730000,35.730000,21038,UBI.PA,-0.001676,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.918148,36.002617,0.084469
2022-04-19 13:00:00+02:00,35.770000,35.900002,35.680000,35.840000,35.840000,9885,UBI.PA,0.003079,35.936000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.892099,35.982290,0.090191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12 10:00:00+02:00,45.790001,45.799999,45.189999,45.250000,45.250000,28059,UBI.PA,-0.011145,45.863999,45.588666,...,0.125333,0.104000,0.320752,1.205128,24.285559,54.651165,0.444374,45.683261,45.641024,-0.042237
2022-08-12 11:00:00+02:00,45.230000,45.490002,45.230000,45.389999,45.389999,15400,UBI.PA,0.003094,45.689999,45.603333,...,0.118667,0.104000,0.179244,1.141027,15.199937,53.293435,0.285212,45.585507,45.609646,0.024138
2022-08-12 12:00:00+02:00,45.380001,45.380001,45.230000,45.340000,45.340000,10428,UBI.PA,-0.001102,45.557999,45.622666,...,0.118667,0.099333,0.223528,1.194632,18.269150,54.434269,0.335619,45.503672,45.575940,0.072268
2022-08-12 13:00:00+02:00,45.320000,45.419998,45.290001,45.389999,45.389999,5281,UBI.PA,0.001103,45.425999,45.648000,...,0.122000,0.096666,0.223528,1.262070,18.269150,55.792708,0.327447,45.465781,45.552697,0.086917


In [126]:
all_data.columns

Index(['open', 'high', 'low', 'close', 'Adj Close', 'volume', 'symbol',
       'return', 'SMA_5', 'SMA_15', 'SMA_ratio', 'SMA5_Volume', 'SMA15_Volume',
       'SMA_Volume_Ratio', 'Lowest_5D', 'High_5D', 'Lowest_15D', 'High_15D',
       'Stochastic_5', 'Stochastic_15', 'Stochastic_%D_5', 'Stochastic_%D_15',
       'Stochastic_Ratio', 'Diff', 'Up', 'Down', 'avg_5up', 'avg_5down',
       'avg_15up', 'avg_15down', 'RS_5', 'RS_15', 'RSI_5', 'RSI_15',
       'RSI_ratio', '5Ewm', '15Ewm', 'MACD'],
      dtype='object')

In [127]:
Dataset_columns = ['SMA_ratio',
                   #'ATR_5','ATR_15','ATR_Ratio',
                   #'ADX_5','ADX_15',
                   'SMA_Volume_Ratio',
                   #'Stochastic_5',
                   'Stochastic_15',
                   'Stochastic_Ratio',
                   #'RSI_5',
                   'RSI_15',
                   'RSI_ratio',
                   'MACD',
                   'close', 'return']

In [128]:
all_data = all_data[Dataset_columns]

In [129]:
all_data = all_data.rename(columns={"return": "price_pct_variation"})
all_data.to_csv('ubisoft_dataset_1h.csv')